In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.utils import np_utils

In [ ]:
# defining the labels in the dataset
labels = ['Walking',
         'Jogging',
         'Sitting',
         'Standing',
         'Upstairs',
         'Downstairs']

In [ ]:
# To read the file and making a dataset 
def read_file(file_path):
    
    # column names for the datasaet
    column_names = ['user', 'activity', 'timestamp', 'x-accel', 'y-accel', 'z-accel']
    dataset = pd.read_csv(file_path, sep=',', header=None, names = column_names)
    dataset['z-accel'].replace(to_replace = r';', value = r'', regex = True)
    dataset['z-accel'] = dataset['z-accel'].astype(float)
    dataset.dropna(axis=0, how='any', inplace=True)
    return dataset

In [ ]:
# To split test and train data
def train_test(dataset):
    #train,test = train_test_split(dataset, test_size = 0.1)
    test = dataset[dataset['user'] > 30]
    train = dataset[dataset['user'] <= 30]

    return train,test

In [ ]:
# TO normalize the data set before performing the operation
def normalize(dataset):
    
    dataset['x-accel'] = dataset['x-accel']/dataset['x-accel'].max()
    dataset['y-accel'] = dataset['y-accel']/dataset['y-accel'].max()
    dataset['z-accel'] = dataset['z-accel']/dataset['z-accel'].max()
    
    return dataset

In [ ]:
def create_segments_and_labels(dataframe, time_steps, step, label_name):

    # x, y, z acceleration as features
    features = 3
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    segments = []
    labels = []
    for i in range(0, len(dataframe) - time_steps, step):
        xs = dataframe['x-accel'].values[i: i + time_steps]
        ys = dataframe['y-accel'].values[i: i + time_steps]
        zs = dataframe['z-accel'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        label = stats.mode(dataframe[label_name][i: i + time_steps])[0][0]
        segments.append([xs, ys, zs])
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, features)
    labels = np.asarray(labels)

    return reshaped_segments, labels

In [ ]:
dataset = read_file('WISDM_ar_v1.1_raw.txt')
column = 'Activity'
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column with encoded values
dataset[column] = le.fit_transform(dataset['activity'].values.ravel())
train,test = train_test(dataset)

print("Total size of Available data set is  " + str(len(dataset)))
print("Total size of Train data set is  " + str(len(train)))
print("Total size of Test data set is  " + str(len(test)))

# create train dataset
x_train, y_train = create_segments_and_labels(train,
                                              time_steps = 80,
                                              step = 40,
                                              label_name = 'Activity')

# create test dataset
x_test, y_test = create_segments_and_labels(test,
                                              time_steps = 80,
                                              step = 40,
                                              label_name = 'Activity')

In [ ]:
num_time_periods, total_axis = x_train.shape[1], x_train.shape[2]
num_classes = le.classes_.size
input_shape = (num_time_periods*total_axis)
x_train = x_train.reshape(x_train.shape[0], input_shape)
x_test = x_test.reshape(x_test.shape[0], input_shape)

print('x_train shape:', x_train.shape)
print('input_shape:', input_shape)
y_train_hot = np_utils.to_categorical(y_train, num_classes)
print('New y_train shape: ', y_train_hot.shape)
y_test_hot = np_utils.to_categorical(y_test, num_classes)
print('New y_test shape: ', y_test_hot.shape)

In [ ]:
def model():
    
    model = Sequential()
    model.add(Reshape((80, 3), input_shape=(input_shape,)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [ ]:
model = model()
print(model.summary())
model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 300
EPOCHS = 25

history = model.fit(x_train,
                      y_train_hot,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      validation_split=0.2,
                      verbose=1)


In [ ]:
y_predict = model.predict(x_test)
y_actual = np.argmax(y_predict, axis=1)
y_expected = np.argmax(y_test_hot, axis=1)

print(y_expected.shape)
print(y_actual.shape)
print(classification_report(y_expected, y_actual))
